In [14]:
import os
from six.moves import urllib
import tarfile


In [20]:
file_path=os.path.join(r'C:\Users\Nplite\Desktop\INEURON\INSURANCE_PREDICTION\Jnotebook','insurance.zip')
dataset_download_url =r'https://drive.google.com/u/0/uc?id=1QuiH3YyUUQRMC0h5T-ictME30_kh1H9x&export=download'
urllib.request.urlretrieve(dataset_download_url,'insurance.zip')

('insurance.zip', <http.client.HTTPMessage at 0x1ceb865d248>)

In [21]:
os.chdir(r'C:\Users\Nplite\Desktop\INEURON\INSURANCE_PREDICTION')

In [22]:
file_path=r'C:\Users\Nplite\Desktop\INEURON\INSURANCE_PREDICTION\config\config.yaml'
from InsurancePremiumPrediction.config.configuration import Configuration
config=Configuration(file_path)
data_ingestion_coonfig_=config.get_data_ingested_config()

In [23]:
from InsurancePremiumPrediction.component.data_ingestion import DataIngestion
x=DataIngestion(data_ingestion_config=data_ingestion_coonfig_)


In [24]:
x.download_insurance_data()

'C:\\Users\\Nplite\\Desktop\\INEURON\\INSURANCE_PREDICTION\\InsurancePremiumPrediction\\artifact\\data_ingestion\\22-10-05-00-04-21\\zip_data\\insurance.zip'

In [25]:
x.extract_zip_file('C:\\Users\\Nplite\\Desktop\\INEURON\\INSURANCE_PREDICTION\\InsurancePremiumPrediction\\artifact\\data_ingestion\\22-10-05-00-04-21\\zip_data\\insurance.zip')

In [26]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

In [27]:
df = pd.read_csv(r'C:\\Users\\Nplite\\Desktop\\INEURON\\INSURANCE_PREDICTION\\InsurancePremiumPrediction\\artifact\\data_ingestion\\22-10-05-00-04-21\\raw_data\\insurance.csv')

In [28]:
df

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86
...,...,...,...,...,...,...,...
1333,50,male,31.0,3,no,northwest,10600.55
1334,18,female,31.9,0,no,northeast,2205.98
1335,18,female,36.9,0,no,southeast,1629.83
1336,21,female,25.8,0,no,southwest,2007.95


In [29]:
df.describe()

,age,bmi,children,expenses
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.665471,1.094918,13270.422414
std,14.049960,6.098382,1.205493,12110.011240
min,18.000000,16.000000,0.000000,1121.870000
25%,27.000000,26.300000,0.000000,4740.287500
50%,39.000000,30.400000,1.000000,9382.030000
75%,51.000000,34.700000,2.000000,16639.915000
max,64.000000,53.100000,5.000000,63770.430000


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   expenses  1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [31]:
df.bmi.describe()

count    1338.000000
mean       30.665471
std         6.098382
min        16.000000
25%        26.300000
50%        30.400000
75%        34.700000
max        53.100000
Name: bmi, dtype: float64

In [32]:
df.expenses.describe()

count     1338.000000
mean     13270.422414
std      12110.011240
min       1121.870000
25%       4740.287500
50%       9382.030000
75%      16639.915000
max      63770.430000
Name: expenses, dtype: float64

In [34]:
df['BMI_Categories'] = pd.cut(df.bmi,bins=[10,20,30,40,50,np.inf],
                                             labels=[1,2,3,4,5])

In [35]:
df

,age,sex,bmi,children,smoker,region,expenses,BMI_Categories
0,19,female,27.9,0,yes,southwest,16884.92,2
1,18,male,33.8,1,no,southeast,1725.55,3
2,28,male,33.0,3,no,southeast,4449.46,3
3,33,male,22.7,0,no,northwest,21984.47,2
4,32,male,28.9,0,no,northwest,3866.86,2
...,...,...,...,...,...,...,...,...
1333,50,male,31.0,3,no,northwest,10600.55,3
1334,18,female,31.9,0,no,northeast,2205.98,3
1335,18,female,36.9,0,no,southeast,1629.83,3
1336,21,female,25.8,0,no,southwest,2007.95,2


In [45]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

In [46]:
for train_index,test_index in split.split(df,df.BMI_Categories):
    train=df.loc[train_index]
    test=df.loc[test_index]

In [47]:
train = train.drop(columns=['BMI_Categories'])
train

,age,sex,bmi,children,smoker,region,expenses
313,49,male,35.9,0,no,southeast,8124.41
552,62,male,21.4,0,no,southwest,12957.12
780,30,male,24.4,3,yes,southwest,18259.22
657,27,male,33.2,2,no,northwest,4058.71
125,26,female,28.8,0,no,northeast,3385.40
...,...,...,...,...,...,...,...
862,55,female,33.5,2,no,northwest,12269.69
69,28,male,24.0,3,yes,southeast,17663.14
1286,28,female,17.3,0,no,northeast,3732.63
509,57,female,28.7,0,no,southwest,11455.28


In [48]:
test = test.drop(columns=['BMI_Categories'])
test

,age,sex,bmi,children,smoker,region,expenses
109,63,male,35.1,0,yes,southeast,47055.53
631,24,male,29.3,0,no,southwest,1977.82
411,44,female,20.2,1,yes,northeast,19594.81
1084,62,female,30.5,2,no,northwest,15019.76
583,32,female,23.7,1,no,southeast,17626.24
...,...,...,...,...,...,...,...
1178,23,female,34.9,0,no,northeast,2899.49
704,47,female,29.5,1,no,northwest,8930.93
889,57,male,33.6,1,no,northwest,11945.13
1001,24,male,32.7,0,yes,southwest,34472.84


In [50]:
import os
os.chdir(r'C:\Users\Nplite\Desktop\INEURON\INSURANCE_PREDICTION')
from InsurancePremiumPrediction.component.data_ingestion import DataIngestion
from InsurancePremiumPrediction.config.configuration import Configuration
file_path=r'C:\Users\Nplite\Desktop\INEURON\INSURANCE_PREDICTION\config\config.yaml'
config=Configuration(file_path)
data_ingestion_config_=config.get_data_ingested_config()


In [51]:
data_ingestion=DataIngestion(data_ingestion_config=data_ingestion_config_)

In [52]:
data_ingestion.initiate_data_ingestion()

InsuranceException: Error occured in script:
        [C:\Users\Nplite\Desktop\INEURON\INSURANCE_PREDICTION\InsurancePremiumPrediction\component\data_ingestion.py] at try block line number :
        [102] and exception block number [106],
        error_message :[Error occured in script:
        [C:\Users\Nplite\Desktop\INEURON\INSURANCE_PREDICTION\InsurancePremiumPrediction\component\data_ingestion.py] at try block line number :
        [29] and exception block number [41],
        error_message :[[WinError 5] Access is denied: 'C:\\Users\\Nplite\\Desktop\\INEURON\\INSURANCE_PREDICTION\\InsurancePremiumPrediction\\artifact\\data_ingestion\\22-10-05-00-04-21\\zip_data']]